In [226]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

print('Libraries Imported.')

Libraries Imported.


In [227]:
# retruns a new web driver object
def createNewDriver():   

    options = webdriver.ChromeOptions() 
    # options.add_argument('--headless') # run driver without browser
    driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                                options=options)

    return driver

In [228]:
# navigate driver to proper frame within Product Line tab
def navigateToProductList(driver):
    url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

    driver.get(url)
    # navigate to product line tab
    frameset = driver.find_element_by_tag_name('frameset')

    frame = frameset.find_element_by_tag_name('frame')
    frame_name = frame.get_attribute('name')

    # grab last 6 chars of frame name since it changes everytime page is loaded
    frame_suffix = frame_name[11:17]
    # print(frame_suffix)

    # set driver in frame that contains necessary info
    driver.switch_to.frame(frame)

    # find product line tab
    product_line = driver.find_element_by_id("Tab2")

    # select product line tab
    product_line.click()
    # print(browser.page_source)

    # set names/IDs of frames based on 6 chars that were previously saved
    prodLineFrameID = 'iframeSearchProdLine_' + frame_suffix
    detailFrameID = 'plDetailFrame'
    dataFrameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains jstree of all products/parts
    frame = driver.find_element_by_id(prodLineFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_id(detailFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_name(dataFrameName)
    driver.switch_to.frame(frame)

    # click into ECHO jstree
    cur_node = driver.find_element_by_id('pn18')
    cur_node.click()

In [229]:
def getNodes(unorderedList):
    # get all list items from unordered list param
    nodes = unorderedList.find_elements_by_tag_name("li")

    # iterate through all list items in current unordered list
    for node in nodes:
        # grab and store node ID
        node_id = node.get_attribute('id')
        # grab and store ID of corresponding anchor
        anchor_id = node_id + '_anchor'

        # make driver wait for node to be clickable before clicking
        wait = WebDriverWait(driver, 10)
        anchor = wait.until(EC.element_to_be_clickable((By.ID, anchor_id))).click()
       
        # (base case) check if node is a parent, if false gather info from leaf node
        if node.get_attribute('data-assembly-is-parent') == "false":
            # TO DO: gather data from part and pull to dataframe?
            print(' Reached leaf node.')
        # else, click the element to expand child list
        else:
            # get xpath of current node
            xpath = '//li[@id = \'' + node_id + '\' and @aria-expanded = \'true\' and @aria-busy = \'false\']'

            # wait for list item to be visible
            wait = WebDriverWait(driver, 10)
            wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))

            # once visible get a new reference to it, as the old reference is most likely stale
            node = driver.find_element_by_xpath(xpath) 

            # find next unordered list element and call getParts recursively to iterate through next level
            getNodes(node.find_element_by_tag_name("ul"))

In [ ]:
def getData(driver):
    frame = driver.find_element_by_name('plDetail')
    driver.switch_to_frame(frame)

    frameset = driver.find_element_by_xpath('frameset')
    driver.switch_to_frame(frameset)
    
    frame = driver.find_element_by_name('IPL694611DATA_GNRVYx')
    driver.switch_to_frame(frame)

    table = driver.find_element_by_class_id('tblIPLMain')

    

In [ ]:
# driver.execute_script("return $('.jstree-open').jstree('open_all');")
# create new webdriver to begin exploring target data
driver = createNewDriver()
navigateToProductList(driver)

# select first ul that contains all subdirectories
productList = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//ul[contains(@class, \'jstree-container-ul\')][li[@aria-level = \'1\']]')))
# pass list of products into getNodes(), a recursive func
getNodes(productList)